# **Virtual generation of low-resolution image pairs**

---

<font size = 4>To train a CARE (3D) model, pairs of low and high-resolution images are necessary. When only high-resolution images are available, this notebook can virtually generate corresponding low-resolution images. It is crucial to ensure that the image pairs maintain identical dimensions throughout this process, as it is a requisite for using the CARE (3D) tool.
Simulating low-quality images that require restoration (low-resolution pair) involves initially scaling down the original images (high-resolution pair). Subsequently, noise is applied to the images, followed by rescaling. This deliberate process induces the necessary loss of quality required for the training process while ensuring the preservation of the mandatory dimensions.

## **1. Dependencies**
---


### **1.1. Install dependencies**
---
<font size = 4>

In [ ]:
#@markdown ##Install dependencies
! pip install -q readlif
! pip install -q SimpleITK
! pip install -q aicsimageio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.7/52.7 MB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.7/138.7 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.6/39.6 MB 44.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 248.4/248.4 kB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 405.2/405.2 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.1/206.1 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.1/195.1 kB 19.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 92.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.4/216.4 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB

### **1.2. Load key dependencies**
---
<font size = 4>

In [ ]:
#@markdown ##Load dependencies

from readlif.reader import LifFile
import numpy as np
import tifffile as tf
import os
import SimpleITK as sitk
from aicsimageio import AICSImage
import shutil
from skimage import io, img_as_uint
from skimage.util import random_noise
import xml.etree.ElementTree as ET
import ipywidgets as widgets
from IPython.display import display

## **2. Initialise the Colab session**
---

### **2.1. Mount your Google Drive**
---
<font size = 4> To use this notebook on the data present in your Google Drive, you need to mount your Google Drive to this notebook.

<font size = 4> Play the cell below to mount your Google Drive and follow the link. In the new browser window, select your drive and select 'Allow', copy the code, paste into the cell and press enter. This will give Colab access to the data on the drive.

<font size = 4> Once this is done, your data are available in the **Files** tab on the top left of notebook.

In [ ]:
#@markdown ##Play the cell to connect your Google Drive to Colab

#@markdown * Follow the instructions.

#@markdown * Click on "Files" site on the right. Refresh the site. Your Google Drive folder should now be available here as "drive".

# mount user's Google Drive to Google Colab.
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


**<font size = 4> If you cannot see your files, reactivate your session by connecting to your hosted runtime.**


<img width="40%" alt ="Example of image detection with retinanet." src="https://github.com/HenriquesLab/ZeroCostDL4Mic/raw/master/Wiki_files/connect_to_hosted.png"><figcaption> Connect to a hosted runtime. </figcaption>

## **3. Select your paths**

---


<font size = 4> **Path and folder path of the input images**

<font size = 4>**`base_path`:** this is the folder path where the folder with the high-resolution images in '.tiff' format is located in your google drive, and where the output will be saved.

<font size = 4>**`input_folder`:** this is the folder path where the high-resolution images in '.tiff' format are.





 To find the paths, go to your Files on the left of the notebook, navigate to the folders containing your files and copy the path by right-clicking on the folder, **Copy path** and pasting it into the right box below.


In [ ]:
#@markdown ##Paths:

# base folder where the GT image is
base_path = "/content/gdrive/MyDrive/Colab Notebooks/res" #@param {type:"string"}

# .tiff images folder name
input_folder = "/content/gdrive/MyDrive/Colab Notebooks/res/train_up" #@param {type:"string"}

## **4. Virtual generation of low-resolution image pairs**
---

<font size = 4>**`down_factor_xy`:** this numerical value acts as a divisor for the original x and y dimensions of the images, allowing for downscaling. It is important to note that after division, only the non-decimal portion determines the new dimensions (no rounding occurs). For example, if the initial x and y dimensions are both 2048, and the rescaling factor is set to 10, the resulting dimensions will be 204. Prior to this step, understanding the precise pixel dimensions of your images is crucial to accurately select the required rescaling factor.

<font size = 4>**`noise_var_range`:**this parameter sets the range for the noise addition and is used to randomly sample a value within this range. We recommend the following range: 0.001, 0.003. Please use this format when specifying the range. Disregard the red warning message shown below this field.

In [ ]:
def tiff_imgs_pairs(base_path, original_images_folder, var_range, down_factor_xy=None, delete_intermediate=False):
    ''' downscale, add noise and upscale .tiff imgs'''

    # Downscale

    # paths
    down_folder_name = 'down'
    output_folder_down = os.path.join(base_path, down_folder_name)
    if down_folder_name not in os.listdir(base_path):
        os.makedirs(output_folder_down)

    file_list = os.listdir(original_images_folder)

    original_size_dict = {}

    for file_name in file_list:
        if file_name.endswith('.tiff'):
            input_image_path = os.path.join(original_images_folder, file_name)
            output_image_path = os.path.join(output_folder_down, file_name)

            # Read image
            img = sitk.ReadImage(input_image_path)

            # Original dimensions [X, Y, Z]
            original_size = img.GetSize()
            original_size_dict[file_name] = original_size

            # Calculate new dimensions [X, Y]
            new_size = [int(sz / down_factor_xy) if idx != 2 else sz for idx, sz in enumerate(original_size)]

            # Calculate scale factor for new spacing
            scale_factor = [old_size / new_size_val for old_size, new_size_val in zip(original_size, new_size)]

            # Calculate new spacing
            new_spacing = [sz * scale_factor_val for sz, scale_factor_val in zip(img.GetSpacing(), scale_factor)]

            # Apply resolution reduction
            resampler = sitk.ResampleImageFilter()
            resampler.SetOutputSpacing(new_spacing)
            resampler.SetSize(new_size)
            resampler.SetOutputDirection(img.GetDirection())
            resampler.SetOutputOrigin(img.GetOrigin())
            resampler.SetTransform(sitk.Transform())
            resampler.SetDefaultPixelValue(img.GetPixelIDValue())

            # Execute resampling
            output_image = resampler.Execute(img)

            # Save images
            sitk.WriteImage(output_image, output_image_path)

    # Gaussian noise

    # Paths
    noise_folder_name = 'noise'
    output_folder_noise = os.path.join(base_path, noise_folder_name)
    if noise_folder_name not in os.listdir(base_path):
        os.makedirs(output_folder_noise)

    file_list = os.listdir(output_folder_down)

    for file_name in file_list:
        # Read the source image from the 'output_folder_down' directory
        input_image_path = os.path.join(output_folder_down, file_name)
        output_image_path = os.path.join(output_folder_noise, file_name)

        img = io.imread(input_image_path)

        # Apply Gaussian noise to the source image
        noise_var = np.random.uniform(var_range[0], var_range[1])
        noisy_img = random_noise(img, mode='gaussian', mean=0.0, var=noise_var, clip=True)

        # Convert images to uint16 format (16 bits per pixel)
        noisy_img_16bit = img_as_uint(noisy_img)

        # Save the .tiff image with LZW compression
        io.imsave(output_image_path, noisy_img_16bit, compression='lzw')


    # Upscale

    # Paths
    up_folder_name = 'train_down'
    output_folder_up = os.path.join(base_path, up_folder_name)
    if up_folder_name not in os.listdir(base_path):
        os.makedirs(output_folder_up)

    file_list = os.listdir(output_folder_noise)

    for file_name in file_list:
        if file_name.endswith('.tiff'):
            input_image_path = os.path.join(output_folder_noise, file_name)
            output_image_path = os.path.join(output_folder_up, file_name)

            # Read image
            img = sitk.ReadImage(input_image_path)

            # Original size based on filename
            original_size = original_size_dict.get(file_name)
            if original_size is None:
                continue  # Skip images without original size info

            # New dimensions to the original size
            new_size = original_size

            # Calculate scale factor for new spacing
            scale_factor = [old_size / new_size_val for old_size, new_size_val in zip(img.GetSize(), new_size)]

            # Calculate new spacing
            new_spacing = [sz * scale_factor_val for sz, scale_factor_val in zip(img.GetSpacing(), scale_factor)]

            # Apply resolution increase
            resampler = sitk.ResampleImageFilter()
            resampler.SetOutputSpacing(new_spacing)
            resampler.SetSize(new_size)
            resampler.SetOutputDirection(img.GetDirection())
            resampler.SetOutputOrigin(img.GetOrigin())
            resampler.SetTransform(sitk.Transform())
            resampler.SetDefaultPixelValue(img.GetPixelIDValue())

            # Execute resampling
            output_image = resampler.Execute(img)

            # Save images
            sitk.WriteImage(output_image, output_image_path)

    # Delete intermediate folders
    if delete_intermediate:
        shutil.rmtree(output_folder_down)
        shutil.rmtree(output_folder_noise)

    return output_folder_up


down_factor_xy = 2 #@param{type:"number"}
noise_var_range = 0.001, 0.003 #@param{type:"number"}

if down_factor_xy == None or noise_var_range == None :
  print('Please enter a value')
else:
# Call function
  output_folder_up = tiff_imgs_pairs(base_path, input_folder, noise_var_range, down_factor_xy, delete_intermediate=True, )
  if output_folder_up:
    print(f'The lower resolution image pairs virtually generated are in {output_folder_up}')


The lower resolution image pairs virtually generated are in /content/gdrive/MyDrive/Colab Notebooks/res/train_down
